# Predicting SAT Scores for NYC Schools

## Goal

Use datasets from [NYC Open Data](https://data.cityofnewyork.us/) to build a unified, clean dataset ready for use with machine learning algorithms to predict student SAT scores on a per school basis.

First, let's ingest the data and get the lay of the land. The datasets are from [NYC Open Data](https://nycopendata.socrata.com/).

In [ ]:
import pandas as pd

# Load the data
dsProgReports = pd.read_csv('NYC_Schools/School_Progress_Reports_-_All_Schools_-_2009-10.csv')
dsDistrict = pd.read_csv('NYC_Schools/School_District_Breakdowns.csv')
dsClassSize = pd.read_csv('NYC_Schools/2009-10_Class_Size_-_School-level_Detail.csv')
dsAttendEnroll = pd.read_csv('NYC_Schools/School_Attendance_and_Enrollment_Statistics_by_District__2010-11_.csv')[:-2] #last two rows are bad
dsSATs = pd.read_csv('NYC_Schools/SAT__College_Board__2010_School_Level_Results.csv') # Dependent

## Outline

Pandas has read the data files with no issues. Next let's create a rough map of where we are going.

We have five datasets here, each with information about either schools or districts. We're going to join all of this information together into a clean data file, with one row for each school, joined with as much information we can gather about that school & its district. This will include the mean SAT scores (Critical Reading, Math, and Writing) for each school in 2010. Those are the "dependent variable" targets for the prediction problem.

Let's look at the datasets.

In [ ]:
dsSATs.info()

dsSATs contains the SAT scores we want to be able to predict along with `School Name` and something called `DBN`. That turns out to be an important key code that we can use to `join` the files.

In [ ]:
dsProgReports.info()

In [ ]:
dsDistrict.info()

In [ ]:
dsClassSize.info()

In [ ]:
dsAttendEnroll.info()

## Joining Strategy

We are going to build a dataset to predict Critical Reading Mean, Mathematics Mean, and Writing Mean for each school (identified by DBN).

After digging around [NYC Open Data](https://nycopendata.socrata.com/) and looking at the file info, we can identify the following join strategy (using SQL-esque pseudocode):

    dsSATS join dsClassSize on dsSATs['DBN'] = dsClassSize['SCHOOL CODE']
    join dsProgReports on dsSATs['DBN'] = dsProgReports['DBN']
    join dsDistrct on dsProgReports['DISTRICT'] = dsDistrict['JURISDICTION NAME']
    join dsAttendEnroll on dsProgReports['DISTRICT'] = dsAttendEnroll['District']

Now that we have the strategy identified at a high level, there are a number of details we have to identify and take care of first.

## Primary Keys - Schools

Before we can join these three datasets together, we need to normalize their primary keys. Below we see the mismatch between the way the DBN (school id) field is represented in the different datasets.

In [ ]:
pd.DataFrame(data=[dsProgReports['DBN'][0:10], dsSATs['DBN'][0:10], dsClassSize['SCHOOL CODE'][0:10]])

We need to normalize the keys to correct this problem.

It looks like we can `join` to `School Code` if we strip the first two characters off the `DBN`s.

In [ ]:
dsProgReports.DBN = dsProgReports.DBN.map(lambda x: x[2:])
dsSATs.DBN = dsSATs.DBN.map(lambda x: x[2:])

Let's check the keys again.

In [ ]:
pd.DataFrame(data=[dsProgReports['DBN'][0:10], dsSATs['DBN'][0:10], dsClassSize['SCHOOL CODE'][0:10]])

## Primary Keys - Districts

We have a similar story with the district international keys. Again, we need to normalize the keys. The only additional complexity here is that `dsProgReports['DISTRICT']` is typed numerically, whereas the other two district keys are typed as string. We will do some type conversions following the key munging.

First, let's look at the key mismatches:

In [ ]:
pd.DataFrame(data=[dsProgReports['DISTRICT'][:10], dsDistrict['JURISDICTION NAME'][:10], dsAttendEnroll['District'][:10]])

### String wrangling

This is going to require some more complex string manipulation. There is a Python library called "re" that helps us work with what are called "regular expressions." Don't worry if this looks odd or difficult. You can learn about regular expressions with a bit of Web searching.

Note the `astype(int)` at the end of these lines to coerce the column to a numeric type.

In [ ]:
import re
dsDistrict['JURISDICTION NAME'] = dsDistrict['JURISDICTION NAME'].map(lambda x: re.match( r'([A-Za-z]*\s)([0-9]*)', x).group(2)).astype(int)
dsAttendEnroll.District = dsAttendEnroll.District.map(lambda x: x[-2:]).astype(int)

Let's check again.

In [ ]:
pd.DataFrame(data=[dsProgReports['DISTRICT'][:10], dsDistrict['JURISDICTION NAME'][:10], dsAttendEnroll['District'][:10]])

## Additional Cleanup

At this point we could do the joins, but there is still messiness in the data. 

First, let's reindex the DataFrames so the semantics come out a bit cleaner.

In [ ]:
dsProgReports = dsProgReports.set_index('DBN')
dsDistrict = dsDistrict.set_index('JURISDICTION NAME')
dsClassSize = dsClassSize.set_index('SCHOOL CODE')
dsAttendEnroll = dsAttendEnroll.set_index('District')
dsSATs = dsSATs.set_index('DBN')

In [ ]:
dsSATs.head()

In [ ]:
dsClassSize.head()

In [ ]:
dsDistrict.head()

Let's take a look at one of our target variables. We see an "s" value which shouldn't be there.

In [ ]:
dsSATs['Critical Reading Mean'].unique()

To filter the "bad" data, we first create a boolean vector mask. 

In [ ]:
mask = dsSATs['Critical Reading Mean'].map(lambda x: x != 's')
mask.sum()

To be thorough, we should also explore the other variables. It turns out that the problem is solved simply by masking on this one variable - trust me!

In [ ]:
dsSATs = dsSATs[mask]
dsSATs['Critical Reading Mean'].unique()

That looks good.

Now let's cast the key fields to integers to that we can do all of the `join`s.

In [ ]:
dsSATs['Number of Test Takers'] = dsSATs['Number of Test Takers'].astype(int)
dsSATs['Critical Reading Mean'] = dsSATs['Critical Reading Mean'].astype(int)
dsSATs['Mathematics Mean'] = dsSATs['Mathematics Mean'].astype(int)
dsSATs['Writing Mean'] = dsSATs['Writing Mean'].astype(int)

## Transforming variables to enable one row per school

dsClassSize will be a many-to-one join with dsSATs because dsClassSize contains multiple entries per school. We need to summarize and build features from this data in order to get one row per school that will join neatly to dsSATs.

Additionally, the data has an irregular format, consisting of a number of rows per school describing different class sizes, then a final row for that school which contains no data except for a number in the final column, SCHOOLWIDE PUPIL-TEACHER RATIO.

We need to extract the SCHOOLWIDE PUPIL-TEACHER RATIO rows, at which point we'll have a regular format and can build variables via aggregate functions. We'll also drop any variables that can't be easily summarized or aggregated and likely have no bearing on the SAT scores (like School Name).

First, let's look at the data.

In [ ]:
dsClassSize.head()

Next we'll extract the Pupil-Teacher Ratio.

In [ ]:
dsPupilTeacher = dsClassSize.filter(['SCHOOLWIDE PUPIL-TEACHER RATIO'])
dsPupilTeacher['SCHOOLWIDE PUPIL-TEACHER RATIO'].unique()

Filter out any missing or negative or zero values for `SCHOOLWIDE PUPIL-TEACHER RATIO`.

In [ ]:
mask = dsPupilTeacher['SCHOOLWIDE PUPIL-TEACHER RATIO'].map(lambda x: x > 0)
dsPupilTeacher = dsPupilTeacher[mask]
dsPupilTeacher['SCHOOLWIDE PUPIL-TEACHER RATIO'].unique()

Drop the original column.

In [ ]:
dsClassSize = dsClassSize.drop('SCHOOLWIDE PUPIL-TEACHER RATIO', axis=1)
dsClassSize.head()

Now drop non-numeric fields that can't be easily summarized or aggregated and likely have no bearing on the SAT scores.

In [ ]:
dsClassSize = dsClassSize.drop(['BORO','CSD','SCHOOL NAME','GRADE ','PROGRAM TYPE',\
'CORE SUBJECT (MS CORE and 9-12 ONLY)','CORE COURSE (MS CORE and 9-12 ONLY)',\
'SERVICE CATEGORY(K-9* ONLY)','DATA SOURCE'], axis=1)
dsClassSize.head()

Now we're ready to create some summary variables for each school and munge the DataFrame so that it has one row per school.

In [ ]:
import numpy as np
grouped = dsClassSize.groupby(level=0)
dsClassSize = grouped.aggregate(np.max).\
    join(grouped.aggregate(np.min), lsuffix=".max").\
    join(grouped.aggregate(np.mean), lsuffix=".min", rsuffix=".mean").\
    join(dsPupilTeacher)

In [ ]:
dsClassSize.head()

## Joining

One final thing before we join - dsProgReports contains distinct rows for separate grade level blocks within one school. For instance one school (one DBN) might have two rows: one for middle school and one for high school. We'll just drop everything that isn't high school.

And finally we can join our data. (Note these are inner joins, so district data get joined to each school in that district. An inner join keeps only those records that appear in both files being joined.)

In [ ]:
mask = dsProgReports['SCHOOL LEVEL*'].map(lambda x: x == 'High School')
dsProgReports = dsProgReports[mask]

In [ ]:
final = dsSATs.join(dsClassSize).\
join(dsProgReports).\
merge(dsDistrict, left_on='DISTRICT', right_index=True).\
merge(dsAttendEnroll, left_on='DISTRICT', right_index=True)

## Additional Cleanup

We should be in a position to build a predictive model for our target variables right away but unfortunately there is still messy data floating around in the dataframe that machine learning algorithms will choke on. A pure feature matrix should have only numeric features, but we can see that isn't the case. However for many of these columns, the right approach is obvious once we've dug in.

In [ ]:
final.dtypes[final.dtypes.map(lambda x: x=='object')]

We'll just drop string columns. If we were going to do serious machine learning here, we could build features from some of these. But we won't do it in BA575.

In [ ]:
final = final.drop(['School Name','SCHOOL','PRINCIPAL','SCHOOL LEVEL*','PROGRESS REPORT TYPE'],axis=1)

Another little cleanup ...

In [ ]:
final['YTD % Attendance (Avg)'] = final['YTD % Attendance (Avg)'].map(lambda x: x.replace("%","")).astype(float)